In [ ]:
# Sources
# https://matheusfacure.github.io/python-causality-handbook/15-Synthetic-Control.html
# https://nbviewer.jupyter.org/github/OscarEngelbrektson/SyntheticControlMethods/blob/master/examples/user_guide.ipynb

In [ ]:
import warnings
import imp
import ascm # get module reference for reload
import copy
import numpy as np
import os 
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf
from ascm.graph import graph_original
from ascm.graph import ascm_dataframe
from ascm.graph import ascm_visualize
from ascm.graph import rmspe_table
from ascm.graph import p_value
from ascm.graph import placebo
from ascm.graph import ascm_limit_donor_pool
from matplotlib import style
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm
from operator import add
from scipy import stats
from scipy.optimize import fmin_slsqp
from scipy.stats import ttest_ind
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression,Ridge
from toolz import reduce, partial
from typing import List


imp.reload(ascm.graph)
warnings.filterwarnings('ignore')
%matplotlib inline
pd.set_option("display.max_columns", 6)
style.use("fivethirtyeight")

# read data file
# pandas read_csv
dirs = os.getcwd() 
files = os.listdir(dirs) 

data = pd.read_excel(r'/Users/jinhyun/Documents/GitHub/Python/UvA projects/Thesis/MSc/data/processed raw data_2.xlsx') 
data = data.loc[ (data.year < 2021) & (data.year >= 1965)]
raw_data = copy.deepcopy(data)# Copy dataframe



# 1. Data processing

In [ ]:
data_gdp = data[
['year','country','GDP (current US$)',
 'Foreign direct investment, net inflows (BoP, current US$)','GDP growth (annual %)','Life expectancy at birth, total (years)'
 ,'Agriculture, forestry, and fishing, value added (% of GDP)'
 ,'Adolescent fertility rate (births per 1,000 women ages 15-19)'
 ,'Inflation, GDP deflator (annual %)'
 ,'School enrollment, primary (% gross)'
 ,'Fertility rate, total (births per woman)'
 , 'Surface area (sq. km)'
 ,'Mortality rate, under-5 (per 1,000 live births)']]

data_gdp_growth = data[
    ['year','country','GDP growth (annual %)','Industry (including construction), value added (% of GDP)',
    'Life expectancy at birth, total (years)','Inflation, GDP deflator (annual %)']]


data_inv = data[
    ['year','country','Gross capital formation (% of GDP)',
       'Industry (including construction), value added (% of GDP)',
       'Adolescent fertility rate (births per 1,000 women ages 15-19)',
       'Inflation, GDP deflator (annual %)',
       'Merchandise trade (% of GDP)',
       'Agriculture, forestry, and fishing, value added (% of GDP)',
       'School enrollment, primary (% gross)']]

data_export = data[
    ['year','country', 'Exports of goods and services (% of GDP)',
       'Industry (including construction), value added (% of GDP)',
       'Adolescent fertility rate (births per 1,000 women ages 15-19)',
       'Inflation, GDP deflator (annual %)',
       'Surface area (sq. km)',
       'Merchandise trade (% of GDP)',
       'Life expectancy at birth, total (years)',
       'School enrollment, primary (% gross)']]


data_import = data[
    ['year','country', 'Imports of goods and services (% of GDP)',
       'Industry (including construction), value added (% of GDP)',
       'Adolescent fertility rate (births per 1,000 women ages 15-19)',
       'Inflation, GDP deflator (annual %)',
       'Surface area (sq. km)',
       'Merchandise trade (% of GDP)',
       'Life expectancy at birth, total (years)',
       'School enrollment, primary (% gross)']]
       
data_fdi = data[
    ['year','country', 'Foreign direct investment, net inflows (BoP, current US$)',
       'Adolescent fertility rate (births per 1,000 women ages 15-19)',
       'Agriculture, forestry, and fishing, value added (% of GDP)',
       'Exports of goods and services (% of GDP)',
       'Fertility rate, total (births per woman)',
       'GDP (current US$)', 'GDP growth (annual %)',
       'Imports of goods and services (% of GDP)',
       'Industry (including construction), value added (% of GDP)',
       'Inflation, GDP deflator (annual %)',
       'Life expectancy at birth, total (years)',
       'Merchandise trade (% of GDP)',
       'Mortality rate, under-5 (per 1,000 live births)',
       'Population density (people per sq. km of land area)',
        'Population, total',
       'School enrollment, primary (% gross)', 'Surface area (sq. km)',
       'Urban population growth (annual %)' ]]

In [ ]:
display(data_gdp)

In [ ]:
display(data_gdp_growth)

In [ ]:
display(data_inv)

In [ ]:
display(data_export)

In [ ]:
display(data_import)

In [ ]:
display(data_fdi)

# Result

## 1. Raw data

In [ ]:
graph_original('European Union','GDP (current US$)', data_gdp)

In [ ]:
graph_original('European Union','GDP growth (annual %)',data_gdp_growth)

In [ ]:
graph_original('European Union','Exports of goods and services (% of GDP)', data_export)

In [ ]:
graph_original('European Union','Imports of goods and services (% of GDP)', data_import)

In [ ]:
graph_original('European Union','Foreign direct investment, net inflows (BoP, current US$)', data_fdi)

In [ ]:
graph_original('European Union','Gross capital formation (% of GDP)', data_inv)

## 1.2 Dataframe of Difference

In [ ]:
ascm_dataframe('European Union','GDP (current US$)', data_gdp, incident= 'QE', incident_year=2009)

In [ ]:
ascm_dataframe('European Union','GDP growth (annual %)', data_gdp_growth, incident= 'QE', incident_year=2009)

In [ ]:
ascm_dataframe('European Union','Exports of goods and services (% of GDP)', data_export, incident= 'QE', incident_year=2009)

In [ ]:
ascm_dataframe('European Union','Imports of goods and services (% of GDP)', data_import, incident= 'QE', incident_year=2009)

In [ ]:
ascm_dataframe('European Union','Gross capital formation (% of GDP)', data_inv, incident= 'QE', incident_year=2009)

In [ ]:
ascm_dataframe('European Union','Foreign direct investment, net inflows (BoP, current US$)', data_fdi, incident= 'QE', incident_year=2009)

## 1.3 Augmented Synthetic Control Method

In [ ]:
ascm_visualize('European Union','GDP (current US$)', data_gdp, incident= 'QE', incident_year=2009, std_scaling=True, ylabel=False)

In [ ]:
ascm_visualize('European Union','GDP growth (annual %)', data_gdp_growth, incident= 'QE', incident_year=2009, std_scaling=True, ylabel=False)

In [ ]:
ascm_visualize('European Union','Exports of goods and services (% of GDP)', data_export, incident= 'QE', incident_year=2009, std_scaling=True, ylabel=False)

In [ ]:
ascm_visualize('European Union','Imports of goods and services (% of GDP)', data_import, incident= 'QE', incident_year=2009, std_scaling=True, ylabel=False)

In [ ]:
ascm_visualize('European Union','Gross capital formation (% of GDP)', data_inv, incident= 'QE', incident_year=2009, std_scaling=True, ylabel=False)

In [ ]:
ascm_visualize('European Union','Foreign direct investment, net inflows (BoP, current US$)', data_fdi, incident= 'QE', incident_year=2009, std_scaling=True, ylabel=False)

## 1.4 RMSPE ratio

In [ ]:
rmspe_table('European Union', 'GDP (current US$)', data_gdp,  incident= 'QE' ,incident_year=2009, outlier = 1)

In [ ]:
rmspe_table('European Union', 'GDP growth (annual %)', data_gdp_growth,  incident= 'QE' ,incident_year=2009, outlier = 1)

In [ ]:
rmspe_table('European Union', 'Exports of goods and services (% of GDP)', data_export,  incident= 'QE' ,incident_year=2009, outlier = 2)

In [ ]:
rmspe_table('European Union', 'Imports of goods and services (% of GDP)', data_import,  incident= 'QE' ,incident_year=2009, outlier = 2)

In [ ]:
rmspe_table('European Union', 'Gross capital formation (% of GDP)', data_inv,  incident= 'QE' ,incident_year=2009, outlier = 0)

In [ ]:
rmspe_table('European Union','Foreign direct investment, net inflows (BoP, current US$)', data_fdi,  incident= 'QE' ,incident_year=2009, outlier = 0)

## 1.5 placebo effect analysis

In [ ]:
placebo('European Union','GDP (current US$)', data_gdp, incident_year = 2009, ylabel="GDP US$ 10 trillion" )

In [ ]:
placebo('European Union','GDP growth (annual %)', data_gdp_growth, incident_year = 2009, ylabel="GDP US$ 10 trillion" )

In [ ]:
placebo('European Union','Exports of goods and services (% of GDP)', data_export, incident_year = 2009, ylabel="GDP US$ 10 trillion" )

In [ ]:
placebo('European Union', 'Imports of goods and services (% of GDP)', data_import, incident_year = 2009, ylabel="GDP US$ 10 trillion" )

In [ ]:
placebo('European Union','Gross capital formation (% of GDP)', data_inv, incident_year = 2009, ylabel="GDP US$ 10 trillion" )

In [ ]:
placebo('European Union','Foreign direct investment, net inflows (BoP, current US$)', data_fdi, incident_year = 2009, ylabel="GDP US$ 10 trillion" )

## 1.6 p-value analysis

In [ ]:
p_value('European Union','GDP (current US$)', data_gdp, incident_year = 2009, detail= False)

In [ ]:
p_value('European Union','GDP growth (annual %)', data_gdp_growth, incident_year = 2009, detail= False)

In [ ]:
p_value('European Union','Exports of goods and services (% of GDP)', data_export, incident_year = 2009, detail= False)

In [ ]:
p_value('European Union','Imports of goods and services (% of GDP)', data_import, incident_year = 2009, detail= False)

In [ ]:
p_value('European Union', 'Gross capital formation (% of GDP)', data_inv,  incident_year = 2009, detail= False)

In [ ]:
p_value('European Union','Foreign direct investment, net inflows (BoP, current US$)', data_fdi,  incident_year = 2009, detail= False)

## 1.7 Limitation  of donor pool test

In [ ]:
ascm_limit_donor_pool('European Union','GDP (current US$)', data_gdp, incident_year = 2009, std_scaling=True, detail = False, ylabel="GDP US$ 10 trillion")

In [ ]:
ascm_limit_donor_pool('European Union','GDP growth (annual %)', data_gdp_growth, incident_year = 2009, std_scaling=True, detail = False, ylabel=False)

In [ ]:
ascm_limit_donor_pool('European Union', 'Exports of goods and services (% of GDP)', data_export,  incident_year = 2009, std_scaling=True, detail = False, ylabel=False)

In [ ]:
ascm_limit_donor_pool('European Union','Imports of goods and services (% of GDP)', data_import,  incident_year = 2009, std_scaling=True, detail = False, ylabel=False)

In [ ]:
ascm_limit_donor_pool('European Union', 'Gross capital formation (% of GDP)', data_inv, incident_year = 2009, std_scaling=True, detail = False, ylabel=False)

In [ ]:
ascm_limit_donor_pool('European Union','Foreign direct investment, net inflows (BoP, current US$)', data_fdi, incident_year = 2009, std_scaling=True, detail = False, ylabel=False)